In [1]:
import os
import cv2
import psutil
from tkinter import Tk
from tkinter.filedialog import askdirectory
from tqdm.notebook import tqdm

p = psutil.Process(os.getpid())
p.cpu_affinity([0, 1, 2, 3, 4, 5, 6, 7])

start_time = [2] #sec
duration = 5 #min
video_fps = 100

# 開啟資料夾對話框，讓使用者選擇資料夾
Tk().withdraw()
foldername = askdirectory(title='選擇資料夾')

# 取得資料夾內的所有影片檔案
video_files = [f for f in os.listdir(foldername) if f.endswith('.MP4')]
# 輸出資料夾設定
output_dir = askdirectory(title='選擇輸出資料夾')

for filename in tqdm(video_files, desc='total_file'):
   # 影片檔案的完整路徑
    filepath = os.path.join(foldername, filename)

    # 讀取影片
    cap = cv2.VideoCapture(filepath)

    # 獲取原始影片的尺寸
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 新的裁切後的尺寸
    new_width = 1300
    new_height = height // 4  # 裁去上四分之一和下四分之一

    # 計算裁切後的上下邊界
    top = 0
    bottom = top + new_height-80
    left = width//6
    right = left+new_width

    new_width = int((right - left)//1.33)
    new_height = int((bottom - top)//1.33)

    # 輸出影片設定
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    filename_without_ext = os.path.splitext(filename)[0]
    output_filename = os.path.join(output_dir, f"{filename_without_ext}_cropped.mp4")
    output = cv2.VideoWriter(output_filename, fourcc, fps, (new_width, new_height), isColor=False)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    end_frame = (60*duration)*video_fps
    now_frames = start_time*video_fps

    gpu_frame = cv2.cuda_GpuMat()

    # 使用 tqdm 顯示進度條
    for now_frames in tqdm(range(end_frame), desc=filename):  # 使用tqdm迭代
        
        ret, frame = cap.read()
    
        # 當讀取完所有幀，退出循環
        if not ret or now_frames >= end_frame :
            break
        
        if now_frames >= end_frame :    
            print('break')

        # 裁切畫面大小
        cropped_frame = frame[top:bottom, left:right]

        # 進行下採樣
        resized_image = cv2.resize(cropped_frame, (new_width, new_height))

        # 進行灰階轉換（可選）
        gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

        # 使用高斯模糊進行影像平滑化
        blurred_image = cv2.GaussianBlur(gray_image, (0, 0), 3)
        
        # 計算Unsharp Mask濾波器
        output_image = cv2.addWeighted(gray_image, 2, blurred_image, -1, 0)

        # # 輸出裁切後的影片
        output.write(output_image)

        # # # 顯示裁切後的畫面
        # cv2.imshow('unsharp_mask', output_image)

        # # 按下'q'鍵退出
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    # 釋放資源
    cap.release()
    output.release() 

cv2.destroyAllWindows()

total_file:   0%|          | 0/6 [00:00<?, ?it/s]

DoB0526a.MP4:   0%|          | 0/30000 [00:00<?, ?it/s]

DoB1222a.MP4:   0%|          | 0/30000 [00:00<?, ?it/s]

DoB1222b.MP4:   0%|          | 0/30000 [00:00<?, ?it/s]

DoB0526b.MP4:   0%|          | 0/30000 [00:00<?, ?it/s]

DoB0526b_2.MP4:   0%|          | 0/30000 [00:00<?, ?it/s]

DoB1219c.MP4:   0%|          | 0/30000 [00:00<?, ?it/s]

In [2]:
import cv2
import tkinter as tk
from tkinter import filedialog
import os

# 創建Tkinter窗口
root = tk.Tk()
root.withdraw()  # 隱藏主窗口

# 選擇影片文件
file_path = filedialog.askopenfilename(title='選擇影片檔案', filetypes=[("Video files", "*.mp4 *.avi *.mkv")])

# 檢查是否選擇了文件
if file_path:
    # 讀取影片
    cap = cv2.VideoCapture(file_path)

    # 確保影片已成功打開
    if not cap.isOpened():
        print("無法打開影片文件")
    else:
        # 確定輸出文件名（你可以自定義後綴）
        base_name = os.path.basename(file_path)
        output_name = os.path.splitext(base_name)[0] + "_output.mp4"

        # 獲取影片的寬度、高度和幀率
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        # 設置輸出影片的編碼器和幀率
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 使用MP4編碼器
        out = cv2.VideoWriter(output_name, fourcc, fps/20, (width, height))

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # 在這裡對frame進行處理（可選）
            # 這只是一個示例，你可以添加自己的處理邏輯

            # 寫入處理後的frame到輸出影片
            out.write(frame)

        # 釋放資源
        cap.release()
        out.release()

        print("影片處理完成，已保存為", output_name)
else:
    print("未選擇影片文件")

# 關閉Tkinter窗口
root.destroy()


影片處理完成，已保存為 DoB1222b_cropped_output.mp4
